# Blackbody

This program shows you how to calcualte the integrated photon and energy fluxes of the BBODY using two different methods.

1.  Integrating the Function in Wolfram Alpha and then passing an energy array to the integrated function.
2.  Using scipy.integrate to numerically integrate the function. This way is more precise.

### The equation for the Blackbody can be found here:


Original Function, before integration: 
    
\begin{equation}
f\left(E\right) = N \times \left[ \frac{(E/1\ \text{keV} )^2}{\exp(E/kT) -1}  \right] dE \ \ \ \ \ \ \ \         
\end{equation}

where $kT$ is the temperature of the blackbody in keV.
This function is normalized to 1 keV to match RMFIT's version.

### My version of this equation:

Original Function, before integration: 
    
    kT:    blackbody temperature in keV.
    N:     normalization
    E:     energy to integrate over, 10 to 10,000 keV


    f(E) = N * ( E**2 / (exp(E/kT)-1) ) dE

# Begin Program

In [1]:
from __future__ import division
import numpy as np
from scipy import integrate

### Constants and Parameters

In [2]:
keVtoerg    = 1.60217657E-9
emin        = 10.0
emax        = 10000.0

pars     = [40.213, 4.05928592E-02]  
# [kT, norm]

def get_parVals():
    pars     = [40.213, 4.05928592E-02]  
    return pars


## Wolfram Alpha Integration of the Blackbody Function:

http://www.wolframalpha.com/input/?i=(x%5E2%2F+(exp(x%2Ft)-1))+dx



When using Wolfram Alpha to integrate, be careful which letters you use for parameters.  Wolfram alpha has some letters set aside to mean something.  If they are used, you will not get the right answer. For example, E stands for exponential. Do NOT use E for energy.

N can be left out of integration. Simply multiply it back on at the end. The more parameters you have, the less likely Wolfram Alpha will calculate the function without a calculation time issue.

    t - kT the blackbody temperature
    N - normalization
    x - energy


In [3]:
## SBPL FUNCTION normalized at 1 keV, as the RMFIT version is. (E/1 keV)^2 on the numerator.
def blackb(engs, flux, *params):
    from mpmath import polylog, exp, log 
    from mpmath import fp
    import time
    
    t       = float(params[0]) # kT
    start_time = time.time()
    
    for i in range(len(engs)-1):
        if engs[i+1] <= (709.666 * t):  # to avoid exp overflow error

            lowIntegral = (-2*(t**3)) * \
            fp.polylog(3, fp.exp(engs[i]/t)) \
            + ( 2 * engs[i] * (t**2) ) * \
            fp.polylog(2, fp.exp(engs[i]/t)) \
            + (engs[i]**2) * t * \
            fp.log(1. - fp.exp(engs[i]/t)) - \
            ((engs[i]**3)/3.)
            
            highIntegral  = (-2*(t**3)) * \
            fp.polylog(3, fp.exp(engs[i+1]/t)) \
            + ( 2 * engs[i+1] * (t**2) ) * \
            fp.polylog(2, fp.exp(engs[i+1]/t)) \
            + (engs[i+1]**2) * t * \
            fp.log(1. - fp.exp(engs[i+1]/t)) - \
            ((engs[i+1]**3)/3.)

            
            val = (highIntegral - lowIntegral)
            flux[i]      = val
    stop_time = time.time() - start_time 
    print('time: %f seconds'%(stop_time))

In [4]:
N      = 5000
engs   = np.logspace(1, 4, N)
flux   = np.zeros(N)

# WILL STORE CALCULATIONS IN FLUX ARRAY.
blackb(engs, flux, *pars)

norm   = pars[-1]
flux_ph = np.sum(flux) * norm

# NO NEED TO MAKE AN ESBPL FUNCTION.  MULTIPLY ENGS BY THE FLUX.
flux_en = np.sum(flux * engs * keVtoerg) * norm

print(
'''
Photon Flux:  %.9f \t photons s^-1 cm^-2
Energy Flux:  %.9e \t ergs s^-1 cm^-2
'''%(flux_ph, flux_en))

time: 4.569011 seconds

Photon Flux:  6270.996012626 	 photons s^-1 cm^-2
Energy Flux:  1.102864420e-03 	 ergs s^-1 cm^-2



/Users/KimiZ/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:19: ComplexWarning: Casting complex values to real discards the imaginary part


## FUNCTION WITHOUT BEING INTEGRATED.

In [5]:
def bbody(energy):
    from numpy import exp
    kT, norm = get_parVals()
    kT     = float(kT)
    N      = float(norm)
    eng    = energy
    if eng <= (709.666 * kT): # to avoid exp overflow error
        return N * ((eng**2) / (exp(eng/kT)-1))
    else:
        return 0

def ebbody(energy):
    kT, norm = get_parVals()
    kT       = float(kT)
    eng      = energy
    if eng <= (709.666 * kT):
        return eng * bbody(eng)
    else:
        return 0
            

In [6]:
Flux_Ph = integrate.quad(bbody, emin, emax, limit=10000)[0]
Flux_En = integrate.quad(ebbody, emin, emax, limit=10000)[0] * keVtoerg

print(
'''
Photon Flux:  %.9f \t photons s^-1 cm^-2
Energy Flux:  %.9e \t ergs s^-1 cm^-2
'''%(Flux_Ph, Flux_En))


Photon Flux:  6270.996010148 	 photons s^-1 cm^-2
Energy Flux:  1.103626529e-03 	 ergs s^-1 cm^-2

